In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
nltk.download('all')
from nltk import ngrams
import re

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

## When vocabulary is small:
Take vocabulary from train + test data.
Initialize the ngram table with all possible n gram combinations.
Update counts based on train data.
Apply smoothing.
Infer on test data.

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Sem 8/NLP/Mammo.csv')
data.head()

,Features,Birads
0,Soft tissue mass lesion (23 x 20 mm) with spic...,4
1,Parenchyma is predominantly FATTY.No distinctl...,1
2,Parenchyma is predominantly GLANDULAR. No dist...,1
3,Parenchyma is predominantly GLANDULAR. No dist...,2
4,Parenchyma is GLANDULAR and FATTY. Small subce...,2


In [ ]:
# data['text'] = data['text'].apply(lambda x: x[9:] if x.startswith('Subject:') else x)
# data.head()

In [ ]:
def preprocess_generation(texts):
  corpus = []
  for text in texts:
    text = text.lower()
    text = re.sub(r'([^\w\s])', r' \1 ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    corpus.append(text)
  return corpus

In [ ]:
corpus = preprocess_generation(data['Features'])

In [ ]:
train_text, test_text = train_test_split(corpus, test_size = 0.2, random_state=42)

In [ ]:
vocab = []
for sentence in corpus:
  for word in sentence.split():
    if word not in vocab:
      vocab.append(word)

print(f'Vocab size: {len(vocab)}')

Vocab size: 316


In [ ]:
def insert_sentence_delimiters(texts, n):
    corpus = []
    for i in range(len(texts)):
      text = texts[i].split()
      text = ['<s>'] * (n-1) + text + ['</s>']
      corpus.append(text)
    return corpus

In [ ]:
from nltk.util import pr
def build_ngram_matrix(corpus, vocab, n):

  vocab_extended = vocab.copy()
  vocab_extended.extend(['<s>','</s>'])

  def get_possible_key_combinations(vocab, n, poss_keys, curr_keys, curr_index):
    if curr_index == n:
      poss_keys.append(tuple(curr_keys.copy()))
      return
    for word in vocab:
      if word == '</s>' or (len(curr_keys) != 0 and curr_keys[-1] != '<s>' and word == '<s>' ):
        continue
      get_possible_key_combinations(vocab, n, poss_keys, curr_keys + [word], curr_index + 1)

  corpus = insert_sentence_delimiters(corpus, n)

  poss_transition_keys = []
  get_possible_key_combinations(vocab_extended, n - 1, poss_transition_keys, [], 0)

  ngram_matrix = {}
  for key in poss_transition_keys:
    ngram_matrix[key] = {}
    for word in vocab_extended:
      ngram_matrix[key][word] = 0

  for sentence in corpus:
    for i in range(len(sentence) - n + 1):
      context = sentence[i:i+n-1]
      target = sentence[i+n-1]
      ngram_matrix[tuple(context)][target] += 1

  if n == 1:
    return dict(list(ngram_matrix.values())[0])
  return ngram_matrix


In [ ]:
def Lidstone_smoothing(models, vocab, i, k):
  ngram_matrix = models[i]
  if i == 1:
    if 0 in ngram_matrix.values():
      for key in ngram_matrix.keys():
        ngram_matrix[key] = ngram_matrix[key] + k
  else:
    for context in ngram_matrix.keys():
      for key in ngram_matrix[context].keys():
        ngram_matrix[context][key] = ngram_matrix[context][key] + k

  return ngram_matrix

In [ ]:
def Kats_Backoff_smoothing(models, vocab, i, k, discount, katz_models):

  # def lower_ngram(katz_models, i, context, target):
  #   if i == 1:
  #     return katz_models[i][target]
  #   elif katz_models[i][context][target] != 0:
  #     return katz_models[i][context][target]
  #   else:
  #     return lower_ngram(katz_models, i - 1, context[1:], target)

  ngram_matrix = models[i]


  if i == 1: # Apply Lidstone smoothing
    if 0 not in ngram_matrix.values():
      return ngram_matrix
    katz_matrix = {}
    for key in ngram_matrix.keys():
      katz_matrix[key] = ngram_matrix[key]

    denom = sum(ngram_matrix.values()) + k * len(ngram_matrix.keys())
    for key in ngram_matrix.keys():
      katz_matrix[key] = (katz_matrix[key] + k)/denom

  else:
    katz_matrix = {}
    for context in ngram_matrix.keys():
      katz_matrix[context] = {}
      for target in ngram_matrix[context].keys():
        katz_matrix[context][target] = 0

    for context in ngram_matrix.keys():
      c_context = sum(ngram_matrix[context].values())
      if 0 not in ngram_matrix[context].values():
        for target in ngram_matrix[context].keys():
          katz_matrix[context][target] = ngram_matrix[context][target]/c_context
        continue
      if c_context == 0:
        for target in ngram_matrix[context].keys():
          # katz_matrix[context][target] = lower_ngram(katz_models, i - 1, context[1:], target)
          if i == 2:
            katz_matrix[context][target] = katz_models[i - 1][target]
          else:
            katz_matrix[context][target] = katz_models[i - 1][context[1:]][target]
      else:
        prob_left_unseen = 1
        denom = 0
        for target in ngram_matrix[context].keys():
          if ngram_matrix[context][target] != 0:
            katz_matrix[context][target] = (ngram_matrix[context][target] - discount)/c_context
            prob_left_unseen -= katz_matrix[context][target]
          else:
            denom += katz_models[i - 1][target] if i==2 else katz_models[i - 1][context[1:]][target]

        alpha = prob_left_unseen/denom
        for target in ngram_matrix[context].keys():
          if ngram_matrix[context][target] == 0:
            katz_matrix[context][target] = alpha * katz_models[i - 1][target] if i==2 else alpha * katz_models[i - 1][context[1:]][target]

  return katz_matrix

In [ ]:
from sklearn.linear_model import LinearRegression


def GoodTuring_smoothing(models, vocab, i, modified_models):
  ngram_matrix = models[i]
  modified_matrix = {}

  if i == 1:

    N = sum(ngram_matrix.values())
    if 0 not in ngram_matrix.values():
      for key in ngram_matrix.keys():
        modified_matrix[key] = ngram_matrix[key]/ N
      return modified_matrix
    for key in ngram_matrix.keys():
      modified_matrix[key] = ngram_matrix[key]

    freq_freq = {}
    for key in ngram_matrix.keys():
      if ngram_matrix[key] not in freq_freq.keys():
        freq_freq[ngram_matrix[key]] = 1
      else:
        freq_freq[ngram_matrix[key]] += 1

    apply_lr = False

    for j in range(1, max(freq_freq.keys()) + 1):
      if j not in freq_freq.keys():
        freq_freq[j] = 0
        apply_lr = True

    if apply_lr:
      x = []
      y = []
      for r, Nr in freq_freq.items():
          if Nr > 0 and r > 0:
              x.append(np.log(r))
              y.append(np.log(Nr))
      lr = LinearRegression()
      lr.fit(np.array(x).reshape(-1,1), y)
      for key in freq_freq.keys():
        if freq_freq[key] == 0:
          freq_freq[key] = lr.predict(np.array(key).reshape(-1,1))[0]

    freq_freq = dict(sorted(freq_freq.items(), key = lambda x : x[0]))


    revised_counts = {}
    for j in range(len(freq_freq.keys()) - 1):
        revised_counts[list(freq_freq.keys())[j]] = float(list(freq_freq.keys())[j+1]) * freq_freq[list(freq_freq.keys())[j+1]] / (freq_freq[list(freq_freq.keys())[j]] * N)

    revised_counts[list(freq_freq.keys())[-1]] = list(freq_freq.keys())[-1]/N

    for key in ngram_matrix.keys():
      modified_matrix[key] = revised_counts[ngram_matrix[key]]

    return modified_matrix
  else:

    for context in ngram_matrix.keys():
      modified_matrix[context] = {}
      for target in ngram_matrix[context].keys():
        modified_matrix[context][target] = 0

    for context in ngram_matrix.keys():
      N = sum(ngram_matrix[context].values())
      if 0 not in ngram_matrix[context].values():
        for target in ngram_matrix[context].keys():
          modified_matrix[context][target] = ngram_matrix[context][target]/N
        continue
      if N == 0:
          for target in ngram_matrix[context].keys():
            modified_matrix[context][target] = 1 / len(ngram_matrix[context].keys())

      else:
        freq_freq = {}
        for key in ngram_matrix[context].keys():
          if ngram_matrix[context][key] not in freq_freq.keys():
            freq_freq[ngram_matrix[context][key]] = 1
          else:
            freq_freq[ngram_matrix[context][key]] += 1

        apply_lr = False

        for j in range(0, max(freq_freq.keys()) + 1):
          if j not in freq_freq.keys():
            freq_freq[j] = 0
            apply_lr = True

        if apply_lr:
          x = []
          y = []
          for r, Nr in freq_freq.items():
              if Nr > 0 and r > 0:
                  x.append(np.log(r))
                  y.append(np.log(Nr))
          lr = LinearRegression()
          lr.fit(np.array(x).reshape(-1,1), y)
          for key in freq_freq.keys():
            if freq_freq[key] == 0:
              freq_freq[key] = np.exp(lr.predict(np.array(np.log(key)).reshape(-1,1))[0])

        freq_freq = dict(sorted(freq_freq.items(), key = lambda x : x[0]))
        # print(freq_freq)

        revised_counts = {}
        for j in range(len(freq_freq.keys()) - 1):
            revised_counts[list(freq_freq.keys())[j]] = float(list(freq_freq.keys())[j+1]) * freq_freq[list(freq_freq.keys())[j+1]]/(freq_freq[list(freq_freq.keys())[j]] * N)
        revised_counts[list(freq_freq.keys())[-1]] = list(freq_freq.keys())[-1] / N
        # print(revised_counts)

        for target in ngram_matrix[context].keys():
          modified_matrix[context][target] = revised_counts[ngram_matrix[context][target]]

        Z = sum(modified_matrix[context].values())
        for target in modified_matrix[context]:
            modified_matrix[context][target] /= Z

  return modified_matrix

In [ ]:
def smoothing(models, vocab, smoothing = 'Lidstone', k = 1, discount = 0.5):
  modified_matrix = {}
  for i in models.keys():
    print(i)
    if smoothing == 'Lidstone':
      models[i] = Lidstone_smoothing(models, vocab, i, k)
    elif smoothing == 'Goodturing':
      modified_matrix[i] = GoodTuring_smoothing(models, vocab, i, modified_matrix)
    else:
      modified_matrix[i] = Kats_Backoff_smoothing(models, vocab, i, k, discount, modified_matrix)

  return modified_matrix

In [ ]:
def compute_perplexity(ngram_matrix, sentences, prob = True):
  n = len(list(ngram_matrix.keys())[-1]) + 1 #ngram
  test_corpus = insert_sentence_delimiters(sentences, n)
  avg = 0
  for sentence in test_corpus:
    log_prob = 0
    for i in range(len(sentence) - n + 1):
      context = sentence[i:i+n-1]
      target = sentence[i+n-1]
      # if sum(ngram_matrix[tuple(context)].values()) == 0:
      #   print(tuple(context))
      if prob:
        log_prob += np.log(ngram_matrix[tuple(context)][target])
        # print(ngram_matrix[tuple(context)][target])
      else:
        log_prob += np.log(ngram_matrix[tuple(context)][target]/sum(ngram_matrix[tuple(context)].values()))
    avg += np.exp(-log_prob/len(sentence))
    break
  return avg/len(test_corpus)

In [ ]:
def build_models(train_text, vocab, n):
  #Given input to build n gram build all n grams starting from unigrams to n-1 grams
  models = {}
  for i in range(1, n+1):
    models[i] = build_ngram_matrix(train_text, vocab, i)
  return models

models = build_models(train_text, vocab, 3)

In [ ]:
models = smoothing(models, vocab, 'Goodturing')

1
2
3


In [ ]:
print(f'Perplexity of bigram model on test data: {compute_perplexity(models[2], test_text)}')
print(f'Perplexity of trigram model on test data: {compute_perplexity(models[3], test_text)}')

Perplexity of bigram model on test data: 0.11005787516748188
Perplexity of trigram model on test data: 0.09559683931937087


## When vocabulary is large
- consider only ngrams occuring in train and test data to initialize the matrix and then count for train data alone

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Sem 8/NLP/emails.csv')
data.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [ ]:
data['text'] = data['text'].apply(lambda x: x[9:] if x.startswith('Subject:') else x)
corpus = preprocess_generation(data['text'])

In [ ]:
vocab = []
for sentence in corpus:
  for word in sentence.split():
    if word not in vocab:
      vocab.append(word)

print(f'Vocab size: {len(vocab)}')

Vocab size: 37384


In [ ]:
train_texts, test_texts = train_test_split(corpus, test_size = 0.2, random_state=42)

In [ ]:
def build_ngram_matrix(corpus, train_corpus, vocab, n):

  vocab_extended = vocab.copy()
  vocab_extended.extend(['<s>','</s>'])

  corpus = insert_sentence_delimiters(corpus, n)
  train_corpus = insert_sentence_delimiters(train_corpus, n)

  ngram_matrix = {}
  if n == 1:
    for sentence in corpus:
      for word in sentence:
        if word not in ngram_matrix.keys():
          ngram_matrix[word] = 0

    for sentence in train_corpus:
      for word in sentence:
        ngram_matrix[word] += 1

    return ngram_matrix

  else:
    for sentence in corpus:
      for i in range(len(sentence) - n + 1):
        context = sentence[i:i+n-1]
        target = sentence[i+n-1]
        if tuple(context) not in ngram_matrix.keys():
          ngram_matrix[tuple(context)] = {}
        if target not in ngram_matrix[tuple(context)].keys():
          ngram_matrix[tuple(context)][target] = 0

    for sentence in train_corpus:
      for i in range(len(sentence) - n + 1):
        context = sentence[i:i+n-1]
        target = sentence[i+n-1]
        ngram_matrix[tuple(context)][target] += 1

    return ngram_matrix

In [ ]:
def build_ngram_model(corpus, train_corpus, vocab, n):
  models = {}
  for i in range(1, n+1):
    models[i] = build_ngram_matrix(corpus, train_corpus, vocab, i)

  return models

models = build_ngram_model(corpus, train_texts, vocab, 3)

In [ ]:
models = smoothing(models, vocab, 'Goodturing')

1
2


In [20]:
print(f'Perplexity of bigram model on test data: {compute_perplexity(models[2], test_texts)}')
print(f'Perplexity of trigram model on test data: {compute_perplexity(models[3], test_texts)}')

Perplexity of bigram model on test data: 0.03261295278401106
Perplexity of trigram model on test data: 0.004478646467614725
